## Download data and source folder (**Do not Modify**)


In [1]:
!pip install gdown
import gdown
import zipfile
url = 'https://drive.google.com/uc?id=1CQdgTOUlY-TUoWZyxtVZxRthBhSuhDVi'
output = 'source.zip'
gdown.download(url, output, quiet=False)
with zipfile.ZipFile(output, "r") as zip_ref:
    zip_ref.extractall('.')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/uc?id=1CQdgTOUlY-TUoWZyxtVZxRthBhSuhDVi
To: /content/source.zip
100%|██████████| 11.8M/11.8M [00:00<00:00, 45.3MB/s]


## Install package for calculating FLOPS (**Do not Modify**)


In [2]:
!pip install pthflops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Necessary Dependencies
- You may import other packages if you want.


In [3]:
import os
import sys
import os.path as osp

from PIL import Image

import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as T

import glob
import matplotlib.pyplot as plt

from pthflops import count_ops

# functions that you downloaded from the first code cell.
# please use this code for seed reset, dataloaders, and test function  
from src.util import reset
# if data.py is changed, please submit that also.
from src.data import get_dataloader
from src.test import test

## Define label names, data directory, device name.


In [4]:
## DO NOT MODIFY FROM HERE
label_names = ['bug', 'electric', 'fighting', 'fire', 'flying', 'grass', 'ground', 'phychic', 'poison', 'water']
data_dir = 'data'
## DO NOT MODIFY UNTIL HERE

## You may modify device name depending on your workspace spec.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Specify the batch size as you want.
batch_size = 2

## **(TO-DO)** DEFINE YOUR `MyModel`
- Please do not change the class name. Let `MyModel` be your classifier class name. 
- Below is just simple example using ResNet18.

In [5]:
######## Define your classification model.  #######
# below model is not valid anymore since it uses ResNet.
class MyModel(nn.Module):
    def __init__(self, dim_output=len(label_names)):
        super().__init__()        
        self.features = nn.Sequential(
            nn.Conv2d(3, 20, kernel_size=(5,5), padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(20, 40, kernel_size=(3,3), padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(40*28*28, dim_output)
        )
        
    def forward(self, img):
        B, C, H, W = img.shape

        out = self.features(img)
        out = torch.flatten(out, 1)
        out = self.classifier(out)    
        return F.softmax(out)
####################################################

## **(Optional)** Make your own loss function.
- You may change `criterion` as you want. 
- But make your custom loss function work without changing below lines, which starts from `model.train()`


In [6]:
def train(model, optimizer, sample):
    ### You may define your own loss function.###
    criterion = nn.CrossEntropyLoss()
    #############################################

    ### make your code work without changing below lines ###
    model.train()    
    
    input = sample['img'].float().to(device)
    label = sample['label'].long().to(device)

    pred = model(input)

    loss = criterion(pred, label)

    num_correct = torch.sum(torch.argmax(pred, dim=-1)==label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(), num_correct.item()
    ##########################################################

## **(TO-DO)** DEFINE YOUR `get_optimizer`
- Please do not change the function name. Let `get_optimizer` be your classifier class name. 
- This function should return the optimizer for optimizing model parameters properly


In [7]:
######## Define your own function for optimizer.  #######
def get_optimizer(model, lr=1e-4, wd=1e-6):
    return optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
##########################################################

## Repeat Training with different 10 random seeds.

*   Do not change `max_epoch` and `num_seeds`.
*   You  may change lines for `MyModel` and `get_optimizer` part if they require additional inputs.


In [8]:
### Do not change below parameters ###
max_epoch = 5
num_seeds = 10
### Do not change above parameters ###


### Make your code work without changing below lines except for "optimizer" part ###
total_best_accu = []

for seed in range(num_seeds):
    # do not erase this random seed initialization part
    reset(seed)

    # get dataloader by spliting train/test data randomly.
    train_loader, test_loader, train_dataset, test_dataset = get_dataloader(data_dir, label_names, batch_size)

    # you may change below line for model definition if your "MyModel" requires more inputs.
    model = MyModel().to(device)

    # you may change below line for optimizer if your "get optimizer" requires more inputs.
    optimizer = get_optimizer(model)

    ###### do not change below lines. Make your code work with below lines. ####
    best_accu = -100
    for epoch in range(max_epoch):
        avg_tr_loss = 0.0
        avg_tr_correct = 0.0
        for sample in train_loader:
            tr_loss, tr_correct = train(model, optimizer, sample)

            avg_tr_loss += tr_loss / len(train_loader)
            avg_tr_correct += tr_correct / len(train_dataset)

        avg_te_correct = 0.0
        for sample in test_loader:
            te_correct = test(model, sample, device)
            avg_te_correct += te_correct / len(test_dataset)
    
        best_accu = max(avg_te_correct, best_accu)

    print('<<<<<[SEED {}] BEST ACCU : {}>>>>>'.format(seed, best_accu))    
    total_best_accu.append(best_accu)
    if seed < num_seeds-1:
        del model, optimizer
    ############################################################################   


<ipython-input-5-cc0bd8744efc>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)


<<<<<[SEED 0] BEST ACCU : 0.5349397590361452>>>>>
<<<<<[SEED 1] BEST ACCU : 0.5493975903614456>>>>>
<<<<<[SEED 2] BEST ACCU : 0.46987951807228945>>>>>
<<<<<[SEED 3] BEST ACCU : 0.5614457831325298>>>>>
<<<<<[SEED 4] BEST ACCU : 0.5228915662650605>>>>>
<<<<<[SEED 5] BEST ACCU : 0.5469879518072295>>>>>
<<<<<[SEED 6] BEST ACCU : 0.5180722891566271>>>>>
<<<<<[SEED 7] BEST ACCU : 0.4771084337349403>>>>>
<<<<<[SEED 8] BEST ACCU : 0.542168674698795>>>>>
<<<<<[SEED 9] BEST ACCU : 0.4698795180722898>>>>>


## Calculate your grade **(Do not Modify)**
- Grade is calculated based on your `average best accuracy` and `FLOPS`.
- Higher accuracy and lower flops make your grade better.

In [9]:
###### do not change below lines #####
num_ops = count_ops(model, torch.rand(1, 3, 112, 112).to(device), verbose=False)

mean_accu = np.mean(total_best_accu)
accu_thres = 0.75
accu_point = min(1, np.exp(-2*(accu_thres-mean_accu)))

flops = num_ops[0]
flops_thres = 2e8
flop_disadvantage = flops_thres / (flops_thres + flops)

print('*'*50)
print('Mean Accuracy :', mean_accu)
print('Accuracy Point:', accu_point)
print('*'*50)
print('Flops: ', flops)
print('Flops Advantage: ', flop_disadvantage)
print('*'*50)

point = accu_point * flop_disadvantage
print('Total Points : ', point)

threshold = 0.5
max_point = 50

if point > threshold:
    grade = max_point
else:
    grade = np.exp(-2*(threshold - point)) * max_point

print('YOUR grade is {} point'.format(grade))

Input size: (1, 3, 112, 112)
43,120,010 FLOPs or approx. 0.04 GFLOPs
**************************************************
Mean Accuracy : 0.5192771084337352
Accuracy Point: 0.6303716057251455
**************************************************
Flops:  43120010
Flops Advantage:  0.8226389921586462
**************************************************
Total Points :  0.5185682624191612
YOUR grade is 50 point


/usr/local/lib/python3.8/dist-packages/pthflops/ops_fx.py:219: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = target(*args, **kwargs)
